In [1]:
import pandas as pd
import numpy as np

## read csv file

In [2]:
df = pd.read_csv('Tweets_cleaned_data.csv')
df.sample(5)

,target,id,date,flag,user,text,text2,text3,hashtags,tokens,tokens_remove_stopwords,length_1
1229582,4,1991400983,Mon Jun 01 06:58:25 PDT 2009,NO_QUERY,sweeetbutterfly,@qutes0911 i know right! work work work work w...,i know right work work work work work but than...,i know right work work work work work but than...,NaN,"['i', 'know', 'right', 'work', 'work', 'work',...","['know', 'right', 'work', 'work', 'work', 'wor...",15
421599,0,2062455424,Sat Jun 06 23:16:52 PDT 2009,NO_QUERY,bright_side,@michaelmillman better than iphone? i can't us...,better than iphone i cant use eitherbecause of...,better than i cant use of my carrier i am resi...,NaN,"['better', 'than', 'i', 'cant', 'use', 'of', '...","['better', 'cant', 'use', 'carrier', 'resistin...",6
1267749,4,1999912360,Mon Jun 01 21:03:03 PDT 2009,NO_QUERY,YourFav_Redhead,"@ginger_swan oh deff, will send more trivia yo...",oh deff will send more trivia your way,oh will send more trivia your way,NaN,"['oh', 'will', 'send', 'more', 'trivia', 'your...","['oh', 'send', 'trivia', 'way']",4
771835,0,2302519819,Tue Jun 23 16:55:32 PDT 2009,NO_QUERY,SaMannequiN,http://twitpic.com/88hzj - my finger is bleedi...,my finger is bleeding profusely,my finger is bleeding profusely,NaN,"['my', 'finger', 'is', 'bleeding', 'profusely']","['finger', 'bleeding', 'profusely']",3
1082107,4,1968664011,Fri May 29 21:55:51 PDT 2009,NO_QUERY,gacconsultants,@randum20 hello randy; appreciate the shout an...,hello randy appreciate the shout and look forw...,hello randy appreciate the shout and look forw...,NaN,"['hello', 'randy', 'appreciate', 'the', 'shout...","['hello', 'randy', 'appreciate', 'shout', 'loo...",8


## Hashtags

### split for fine hashtage

In [3]:
import re
def split_hashtag(hashtag):
    return re.findall(r"#(\w+)", hashtag)

df['hashtags'] = df['text'].apply(split_hashtag)

In [4]:
# make hashtags in each tweet not duplicated
df['hashtags'] = df['hashtags'].apply(lambda x: list(set(x)))

In [5]:
# show where have hashtags
df['hashtags'].apply(lambda x: len(x) > 0).value_counts()

False    1564153
True       35847
Name: hashtags, dtype: int64

In [6]:
# show the number of hashtags in each tweet
df['num_hashtags'] = df['hashtags'].apply(lambda x: len(x))

### look hashtags

In [7]:
# create an array of hashtags
hashtags = np.array([hashtag for hashtag_list in df['hashtags'][pd.notnull(df['hashtags'])] for hashtag in hashtag_list])

In [8]:
len(hashtags)

41225

In [9]:
# count the number of each hashtag
from collections import Counter
hashtag_counts = Counter(hashtags)

# show the top 10 hashtags
hashtag_counts.most_common(10)


[('followfriday', 2620),
 ('fb', 1786),
 ('ff', 973),
 ('squarespace', 924),
 ('1', 516),
 ('iranelection', 516),
 ('musicmonday', 421),
 ('fail', 360),
 ('2', 335),
 ('asot400', 331)]

In [10]:
# if want all hashtags, and sort max -> min
#hashtag_counts.most_common(len(hashtag_counts))

In [11]:
# create a dataframe of hashtags
hashtag_df = pd.DataFrame(hashtag_counts.most_common(len(hashtag_counts)), columns=['hashtag', 'count'])

In [12]:
# save the dataframe to csv file
# hashtag_df.to_csv('hashtag_counts.csv', index=False)

## Sentiment

In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import tensorflow as tf


c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Instantiate Model

In [14]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
## cardiffnlp/twitter-roberta-base-sentiment


In [15]:
df_sentiment = pd.DataFrame()
df_sentiment = df

In [16]:
df_sentiment.head()

,target,id,date,flag,user,text,text2,text3,hashtags,tokens,tokens_remove_stopwords,length_1,num_hashtags
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - awww, t...",awww thats a bummer you shoulda got david carr...,thats a bummer you got carr of third day to do...,[],"['thats', 'a', 'bummer', 'you', 'got', 'carr',...","['thats', 'bummer', 'got', 'carr', 'third', 'd...",6,0
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his facebook by ...,is upset that he cant update his facebook by t...,is upset that he cant update his by it and mig...,[],"['is', 'upset', 'that', 'he', 'cant', 'update'...","['upset', 'cant', 'update', 'might', 'cry', 'r...",10,0
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@kenichan i dived many times for the ball. man...,i dived many times for the ball managed to sav...,i many times for the ball to save the rest go ...,[],"['i', 'many', 'times', 'for', 'the', 'ball', '...","['many', 'times', 'ball', 'save', 'rest', 'go']",6,0
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,my whole body itchy and like its on fire,[],"['my', 'whole', 'body', 'itchy', 'and', 'like'...","['whole', 'body', 'itchy', 'like', 'fire']",5,0
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",no its not behaving at all im mad why am i her...,no its not at all mad why am i here because i ...,[],"['no', 'its', 'not', 'at', 'all', 'mad', 'why'...","['mad', 'cant', 'see']",3,0


In [17]:
df_sentiment.shape

(1600000, 13)

#### Encode and Calculate Sentiment

In [18]:
def sentiment_score(text):
    if isinstance(text, str):
        # If input is a single string, tokenize it and encode it as a tensor
        tokens = tokenizer.encode(text, return_tensors='pt')
    elif isinstance(text, list):
        # If input is a list of strings, tokenize each string and concatenate the tensors
        tokens = torch.cat([tokenizer.encode(t, return_tensors='pt') for t in text])
    else:
        return -1
        
    result = model(tokens)
    # return int(torch.argmax(result.logits))+1
    # return mean of all the scores
    return int(torch.argmax(result.logits))+1

In [19]:
df_sentiment.head()

,target,id,date,flag,user,text,text2,text3,hashtags,tokens,tokens_remove_stopwords,length_1,num_hashtags
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - awww, t...",awww thats a bummer you shoulda got david carr...,thats a bummer you got carr of third day to do...,[],"['thats', 'a', 'bummer', 'you', 'got', 'carr',...","['thats', 'bummer', 'got', 'carr', 'third', 'd...",6,0
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his facebook by ...,is upset that he cant update his facebook by t...,is upset that he cant update his by it and mig...,[],"['is', 'upset', 'that', 'he', 'cant', 'update'...","['upset', 'cant', 'update', 'might', 'cry', 'r...",10,0
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@kenichan i dived many times for the ball. man...,i dived many times for the ball managed to sav...,i many times for the ball to save the rest go ...,[],"['i', 'many', 'times', 'for', 'the', 'ball', '...","['many', 'times', 'ball', 'save', 'rest', 'go']",6,0
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,my whole body itchy and like its on fire,[],"['my', 'whole', 'body', 'itchy', 'and', 'like'...","['whole', 'body', 'itchy', 'like', 'fire']",5,0
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",no its not behaving at all im mad why am i her...,no its not at all mad why am i here because i ...,[],"['no', 'its', 'not', 'at', 'all', 'mad', 'why'...","['mad', 'cant', 'see']",3,0


In [20]:
df_sentiment = df_sentiment.dropna(subset=['text3'])
# df_sentiment = df_sentiment.sample(frac=0.2).reset_index(drop=True)

In [21]:
df_sentiment.head()

,target,id,date,flag,user,text,text2,text3,hashtags,tokens,tokens_remove_stopwords,length_1,num_hashtags
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - awww, t...",awww thats a bummer you shoulda got david carr...,thats a bummer you got carr of third day to do...,[],"['thats', 'a', 'bummer', 'you', 'got', 'carr',...","['thats', 'bummer', 'got', 'carr', 'third', 'd...",6,0
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his facebook by ...,is upset that he cant update his facebook by t...,is upset that he cant update his by it and mig...,[],"['is', 'upset', 'that', 'he', 'cant', 'update'...","['upset', 'cant', 'update', 'might', 'cry', 'r...",10,0
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@kenichan i dived many times for the ball. man...,i dived many times for the ball managed to sav...,i many times for the ball to save the rest go ...,[],"['i', 'many', 'times', 'for', 'the', 'ball', '...","['many', 'times', 'ball', 'save', 'rest', 'go']",6,0
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,my whole body itchy and like its on fire,[],"['my', 'whole', 'body', 'itchy', 'and', 'like'...","['whole', 'body', 'itchy', 'like', 'fire']",5,0
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",no its not behaving at all im mad why am i her...,no its not at all mad why am i here because i ...,[],"['no', 'its', 'not', 'at', 'all', 'mad', 'why'...","['mad', 'cant', 'see']",3,0


In [22]:
df_sentiment.shape

(1583226, 13)

In [23]:
df_sentiment['sentiment_score'] = df_sentiment['text3'].apply(lambda x: sentiment_score(x))

C:\Users\USER\AppData\Local\Temp\ipykernel_2016\3772641994.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sentiment['sentiment_score'] = df_sentiment['text3'].apply(lambda x: sentiment_score(x))


In [24]:
df_sentiment.shape

(1583226, 14)

In [25]:
df_sentiment.head(10)

,target,id,date,flag,user,text,text2,text3,hashtags,tokens,tokens_remove_stopwords,length_1,num_hashtags,sentiment_score
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - awww, t...",awww thats a bummer you shoulda got david carr...,thats a bummer you got carr of third day to do...,[],"['thats', 'a', 'bummer', 'you', 'got', 'carr',...","['thats', 'bummer', 'got', 'carr', 'third', 'd...",6,0,3
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his facebook by ...,is upset that he cant update his facebook by t...,is upset that he cant update his by it and mig...,[],"['is', 'upset', 'that', 'he', 'cant', 'update'...","['upset', 'cant', 'update', 'might', 'cry', 'r...",10,0,1
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@kenichan i dived many times for the ball. man...,i dived many times for the ball managed to sav...,i many times for the ball to save the rest go ...,[],"['i', 'many', 'times', 'for', 'the', 'ball', '...","['many', 'times', 'ball', 'save', 'rest', 'go']",6,0,2
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,my whole body itchy and like its on fire,[],"['my', 'whole', 'body', 'itchy', 'and', 'like'...","['whole', 'body', 'itchy', 'like', 'fire']",5,0,2
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",no its not behaving at all im mad why am i her...,no its not at all mad why am i here because i ...,[],"['no', 'its', 'not', 'at', 'all', 'mad', 'why'...","['mad', 'cant', 'see']",3,0,1
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@kwesidei not the whole crew,not the whole crew,not the whole crew,[],"['not', 'the', 'whole', 'crew']","['whole', 'crew']",2,0,2
6,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,need a hug,need a hug,need a hug,[],"['need', 'a', 'hug']","['need', 'hug']",2,0,5
7,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@loltrish hey long time no see! yes.. rains a...,hey long time no see yes rains a bit only a bi...,hey long time no see yes a bit only a bit fine...,[],"['hey', 'long', 'time', 'no', 'see', 'yes', 'a...","['hey', 'long', 'time', 'see', 'yes', 'bit', '...",9,0,4
8,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@tatiana_k nope they didn't have it,nope they didnt have it,nope they didnt have it,[],"['nope', 'they', 'didnt', 'have', 'it']","['nope', 'didnt']",2,0,1
9,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?,que me muera,me,[],['me'],[],0,0,3


In [26]:
# show the number of each sentiment score
df_sentiment['sentiment_score'].value_counts()

5    562174
1    430789
3    287646
4    159422
2    143195
Name: sentiment_score, dtype: int64

In [27]:
# save the dataframe to csv file
df_sentiment.to_csv('Tweets_sentiment_16M.csv', index=False)

In [28]:
# sort the dataframe by sentiment score
# df_sentiment.sort_values(by=['sentiment_score'], inplace=True, ascending=False)
# df_sentiment.head(10)